<a href="https://colab.research.google.com/github/BrunoReis136/AI_features/blob/main/ai_video_transform.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 🧰 Instalar dependências
!pip install diffusers transformers accelerate scipy safetensors opencv-python moviepy

# 🧱 Importações
import os
import cv2
from moviepy.editor import *
from PIL import Image
from diffusers import StableDiffusionControlNetPipeline, ControlNetModel, UniPCMultistepScheduler
from transformers import AutoTokenizer
import torch

# 📁 Criar diretórios
os.makedirs("frames_original", exist_ok=True)
os.makedirs("frames_transformados", exist_ok=True)

# 📽️ Upload do vídeo
from google.colab import files
uploaded = files.upload()
video_path = list(uploaded.keys())[0]

# 🎞️ Extrair frames
vidcap = cv2.VideoCapture(video_path)
success, image = vidcap.read()
count = 0
while success:
    cv2.imwrite(f"frames_original/frame{count:03d}.png", image)
    success, image = vidcap.read()
    count += 1
print(f"{count} frames extraídos.")

# 📦 Baixar modelo ControlNet
from diffusers.utils import load_image
controlnet = ControlNetModel.from_pretrained(
    "lllyasviel/sd-controlnet-canny", torch_dtype=torch.float16
)

pipe = StableDiffusionControlNetPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5",
    controlnet=controlnet,
    safety_checker=None,
    torch_dtype=torch.float16
).to("cuda")
pipe.scheduler = UniPCMultistepScheduler.from_config(pipe.scheduler.config)

# 🎨 Função para transformar cada frame
from PIL import ImageFilter

prompt = input("\n Descreva as mudanças no vídeo: \n")
negative_prompt = "blurry, low quality, deformed, text, watermark"

for i in range(count):
    img = Image.open(f"frames_original/frame{i:03d}.png")
    img = img.resize((512, 512))  # redimensiona para SD
    img = img.filter(ImageFilter.EDGE_ENHANCE)  # ou canny se quiser
    output = pipe(prompt, image=img, num_inference_steps=25, negative_prompt=negative_prompt).images[0]
    output.save(f"frames_transformados/frame{i:03d}.png")

# 🎞️ Juntar os frames transformados em vídeo
image_files = [f"frames_transformados/frame{i:03d}.png" for i in range(count)]
clip = ImageSequenceClip(image_files, fps=vidcap.get(cv2.CAP_PROP_FPS))
clip.write_videofile("video_sinistro.mp4", codec="libx264")
